In [1]:
import pandas as pd
from tqdm import trange
from tqdm import tqdm
import tweepy
import pickle
import networkx as nx 

In [2]:
congress = pd.read_pickle('../Data/Interim/congress.pkl')

In [8]:
trump = pd.read_pickle('../Data/Interim/trump.pkl')

In [12]:
# Concatinating the congress with Trump
congress_tweets = pd.concat([congress, trump])

In [13]:
congress_tweets.shape

(2850576, 10)

In [14]:
# Removing duplicates
congress_tweets.drop_duplicates(keep='first', inplace=True)

In [15]:
congress_tweets.shape

(2192037, 10)

In [16]:
# Getting handles
medias = pd.read_table('../Data/Raw/LargestMedia.csv', sep=';')
twitter_handles = pd.read_table('../Data/Processed/Twitter_Handles_updated.csv', sep = ',')

In [17]:
medias['State'] = None
medias['Party'] = None
medias['Type'] = 'Media'
medias.rename(columns={'Twitter name': 'Twitter', 'Media': 'Name'}, inplace=True)

In [19]:
Data_Full = pd.concat([twitter_handles, medias])

In [21]:
congress_tweets[congress_tweets['retweet'] != False].shape

(529703, 10)

In [22]:
retweets_pd = congress_tweets[congress_tweets['retweet'] != False]

In [23]:
#retweets_pd['retweet'].iloc[1] in Data_Full['Twitter']

Data_Full[Data_Full['Twitter'] == 'KamalaHarris']
#[i for i in retweets_pd['retweet'] if i.isin(Data_Full['Twitter'])]


,Twitter,State,Party,Type,Name
117,KamalaHarris,CA,Democrat,Senator,Kamala Harris


In [24]:
retweets_pd = retweets_pd[retweets_pd['retweet'].isin(Data_Full.Twitter)]

In [25]:
retweets_pd.shape

(95139, 10)

In [26]:
# Who retweets who
retweets_pd.head()

,user_id,user_name,id,created_at,text,retweet,retweet_count,favorite_count,in_reply_to_status_id,in_reply_to_user_id
787078,29501253,Adam Schiff,824411350420127745,2017-01-26 00:19:17,Executive orders to break up families in Calif...,KamalaHarris,757,0,NaN,NaN
787222,41417564,Rep. Lloyd Smucker,824583733693075457,2017-01-26 11:44:16,We will use the Congressional Review Act to re...,GOPLeader,230,0,NaN,NaN
787232,2863006655,Brenda Lawrence,824598117857292290,2017-01-26 12:41:26,A day that I will always remember. #tbt #resis...,RepBonnie,12,0,NaN,NaN
787242,231510077,Joaquin Castro,824608733456138243,2017-01-26 13:23:37,Chaffetz: I’m looking into Trump D.C. hotel le...,politico,117,0,NaN,NaN
787249,3353670647,Archive: Dan Donovan,824616666818629632,2017-01-26 13:55:08,The #GOPretreat is about bringing the @HouseGO...,cathymcmorris,21,0,NaN,NaN


In [27]:
df_retweets = pd.DataFrame(data = 0, index = Data_Full['Twitter'], columns = Data_Full['Twitter'])

In [28]:
twitter_name_dict = {name: twitter_handle for twitter_handle, name in zip(Data_Full['Twitter'], Data_Full['Name'])}

In [29]:
for idx, retweet in tqdm(retweets_pd.iterrows(), total=len(retweets_pd)):
    row = twitter_name_dict[retweet['user_name']]
    col = retweet['retweet']

    df_retweets.loc[row, col] += 1



0%|          | 1/95139 [00:00<24:33:13,  1.08it/s]


KeyError: 'Rep. Lloyd Smucker'

In [30]:
twitter_handles

,Twitter,State,Party,Type,Name
0,RepTomPrice,AS,Republican,Representative,Aumua Radewagen
1,JeffFlake,AZ,Republican,Senator,Jeff Flake
2,SenJonKyl,AZ,Republican,Senator,Jon Kyl
3,reppeteaguilar,CA,Democrat,Representative,Peter Aguilar
4,repcardenas,CA,Democrat,Representative,Tony Cardenas
...,...,...,...,...,...
612,MikeforWI,WI,Republican,Representative,Mike Gallagher
613,SenatorEnzi,WY,Republican,Senator,Mike Enzi
614,SenJohnBarrasso,WY,Republican,Senator,John Barrasso
615,Liz_Cheney,WY,Republican,Representative,Liz Cheney
